## Machine Learning Record Mining

Project to create a pipeline that uses GeoDeepDive's output to find Unaquired Sites for Neotoma.

Using NLP parsed text and a Data Science approach, identify whether a paper is suitable for Neotoma and detect features such as 'Site Name', 'Location', 'Age Span' and 'Site Descriptions'.

In [143]:
# Loading libraries

import numpy as np
import pandas as pd
import csv
import psycopg2
import re

# Load Postgres Server
from src.config import config

In [234]:
# Options for DF display
pd.set_option('display.max_colwidth', 300)
pd.set_option('display.max_rows', 10)

#pd.set_option('display.max_rows', None)
#pd.set_option('display.max_columns', None)
#pd.set_option('display.width', None)
#pd.set_option('display.max_colwidth', -1)

## Loading and viewing the Data

### Loading NLP Sentences

In [150]:
# Connect to PostgreSQL server from terminal:
# pg_ctl -D PSQL_Data -l logfile start

try:
    params = config()
    # Connect to the PostgreSQL database
    conn = psycopg2.connect(**params)
    # Create a new cursor
    cur = conn.cursor()
    nlp_sentences = pd.read_sql_query('''SELECT * FROM sentences;''', conn)
    # Close the cursor and connection to so the server can allocate
    # bandwidth to other requests
    cur.close()
    conn.close()
    print('Data obtained from PostgreSQL')
    
# If no SQL db, load from a file
except:
    header_list = ["_gddid", "sentid", "wordidx", "words", "part_of_speech", "special_class", 
               "lemmas", "word_type", "word_modified"]
    nlp_sentences = pd.read_csv("../Do_not_commit_data/sentences_nlp352", sep='\t', names = header_list)
    nlp_sentences = nlp_sentences.replace('"', '', regex = True)\
                                 .replace('\{', '', regex = True)\
                                 .replace('}', '', regex = True)\
                                 .replace(',', ',', regex = True)
    nlp_sentences['wordidx']= nlp_sentences['wordidx'].str.split(",")
    nlp_sentences['words']= nlp_sentences['words'].str.split(",")
    nlp_sentences['poses']= nlp_sentences['poses'].str.split(",")
    nlp_sentences['ners']= nlp_sentences['ners'].str.split(",")
    nlp_sentences['lemmas']= nlp_sentences['lemmas'].str.split(",")
    nlp_sentences['dep_paths']= nlp_sentences['dep_paths'].str.split(",")
    nlp_sentences['dep_parents']= nlp_sentences['dep_parents'].str.split(",")
    print('Data obtained from text file')

Data obtained from PostgreSQL


In [155]:
nlp_sentences.head(10)

,_gddid,sentid,wordidx,words,part_of_speech,special_class,lemmas,word_type,word_modified
0,54b43266e138239d8684efed,1,"[1, 2, 3, 4, 5, 6, 7, 8, 9...","[Available, online, at, ww...","[JJ, NN, IN, NNP, NNP, NNP...","[O, O, O, O, O, O, NUMBER,...","[available, online, at, ww...","[dep, dep, dep, dep, dep, ...","[218, 218, 218, 218, 218, ..."
1,54b43266e138239d8684efed,2,"[1, 2, 3, 4, 5, 6, 7, 8, 9...","[The, Chihuahueños, Bog, r...","[DT, NNP, NN, NN, VBZ, TO,...","[O, O, O, O, O, O, O, NUMB...","[the, Chihuahueños, bog, r...","[det, compound, compound, ...","[4, 4, 4, 5, 0, 11, 11, 11..."
2,54b43266e138239d8684efed,3,"[1, 2, 3, 4, 5, 6, 7, 8, 9...","[An, Artemisia, steppe, ,,...","[DT, NNP, NN, ,, RB, DT, J...","[O, O, O, O, O, O, O, LOCA...","[a, Artemisia, steppe, ,, ...","[det, compound, nsubj, , a...","[3, 3, 10, 0, 9, 9, 9, 9, ..."
3,54b43266e138239d8684efed,4,"[1, 2, 3, 4, 5, 6, 7, 8, 9...","[C/N, ratios, ,, δ13C, and...","[JJ, NNS, ,, NN, CC, NN, N...","[O, O, O, O, O, O, O, O, O...","[c/n, ratio, ,, δ13c, and,...","[amod, compound, , conj:an...","[7, 7, 0, 2, 2, 2, 8, 0, 1..."
4,54b43266e138239d8684efed,5,"[1, 2, 3, 4, 5, 6, 7, 8, 9...","[Higher, percentages, of, ...","[JJR, NNS, IN, JJ, NN, CC,...","[O, O, O, O, O, O, O, O, O...","[higher, percentage, of, a...","[amod, nsubj, case, amod, ...","[2, 10, 5, 5, 2, 5, 9, 9, ..."
5,54b43266e138239d8684efed,294,"[1, 2, 3, 4, 5, 6]","[Anderson, ,, R.S., ,, 198...","[NNP, ,, NNP, ,, CD, .]","[PERSON, O, PERSON, O, DAT...","[Anderson, ,, R.S., ,, 198...","[compound, , , , amod, ]","[3, 0, 0, 0, 3, 0]"
6,54b43266e138239d8684efed,6,"[1, 2, 3, 4, 5, 6, 7, 8, 9...","[From, ca., 8500, to, 6400...","[IN, FW, CD, TO, CD, JJ, N...","[O, O, NUMBER, O, NUMBER, ...","[from, ca., 8500, to, 6400...","[case, nmod:from, compound...","[2, 19, 5, 5, 8, 8, 8, 19,..."
7,54b43266e138239d8684efed,7,"[1, 2, 3, 4, 5, 6, 7, 8, 9...","[C/N, ratios, declined, to...","[JJ, NNS, VBD, TO, PRP$, J...","[O, O, O, O, O, O, O, O, O...","[c/n, ratio, decline, to, ...","[amod, nsubj, , case, nmod...","[2, 3, 0, 8, 8, 8, 8, 3, 0..."
8,54b43266e138239d8684efed,8,"[1, 2, 3, 4, 5, 6, 7, 8, 9...","[Wetter, conditions, retur...","[JJ, NNS, VBD, IN, CD, JJ,...","[O, O, O, O, DATE, O, DURA...","[wetter, condition, return...","[amod, nsubj, , case, numm...","[2, 3, 0, 8, 8, 8, 8, 3, 0..."
9,54b43266e138239d8684efed,9,"[1, 2, 3, 4, 5, 6, 7, 8, 9...","[Higher, charcoal, influx,...","[JJR, NN, NN, NNS, IN, CD,...","[O, O, O, O, O, DATE, O, D...","[higher, charcoal, influx,...","[amod, compound, compound,...","[4, 4, 4, 11, 9, 9, 9, 9, ..."


### Loading Bibliography Data

In [156]:
# TODO Load into SQL server and connect through SQL

bibliography = pd.read_json (r'../Do_not_commit_data/bibjson')

In [157]:
# TODO Figure out how to flatten nested Json `[{}]` 

bibliography.head(10)

,publisher,title,journal,author,year,number,volume,link,_gddid,identifier,type,pages
0,Elsevier,"Palaeoclimate, chronology ...",{'name': {'name': 'Quatern...,"[{'name': 'Huntley, B.'}, ...",1999,7,18,[{'url': 'http://www.scien...,550453fde1382326932d85f7,"[{'type': 'doi', 'id': '10...",article,945--960
1,Canadian Science Publishing,Holocene history of forest...,{'name': {'name': 'Canadia...,"[{'name': 'Bennett, K. D.'}]",1992,1,70,[{'url': 'http://www.nrcre...,578b5aabcf58f1587003efba,"[{'type': 'doi', 'id': '10...",article,6--18
2,Elsevier,Glacial sequence and envir...,{'name': {'name': 'Palaeog...,"[{'name': 'Van Der Hammen,...",1980,NaN,NaN,[{'url': 'http://www.scien...,54b43244e138239d868493cd,"[{'type': 'doi', 'id': '10...",article,247--340
3,GSA,A record of sustained preh...,{'name': {'name': 'Geology'}},"[{'name': 'Munoz, S. E.'},...",2014,6,42,[{'url': 'http://dx.doi.or...,57c5b941cf58f1338eaddb5b,"[{'type': 'doi', 'id': '10...",article,499--502
4,Elsevier,Plant and insect fossils a...,{'name': {'name': 'Quatern...,"[{'name': 'Ashworth, Allan...",1981,1,16,[{'url': 'http://www.scien...,54b43269e138239d8684f87f,"[{'type': 'doi', 'id': '10...",article,66--79
5,Taylor and Francis,10. Na Bahně (Czech Republ...,{'name': {'name': 'Grana'}},"[{'name': 'Pokorný, Petr'}...",2010,1,49,[{'url': 'http://www.tandf...,58d27c30cf58f121bb4ff5db,"[{'type': 'doi', 'id': '10...",article,79--81
6,Canadian Science Publishing,Age verification of the La...,{'name': {'name': 'Canadia...,"[{'name': 'Lowell, Thomas ...",1999,3,36,[{'url': 'http://www.nrcre...,574629c5cf58f15d3f588144,"[{'type': 'doi', 'id': '10...",article,383--393
7,Elsevier,Holocene vegetation histor...,{'name': {'name': 'Quatern...,"[{'name': 'Lucas, Jennifer...",2013,3,79,[{'url': 'http://www.scien...,54b43265e138239d8684ee17,"[{'type': 'doi', 'id': '10...",article,366--376
8,Elsevier,Synchronization of late-gl...,{'name': {'name': 'Quatern...,"[{'name': 'Gonzales, Leila...",2009,2,72,[{'url': 'http://www.scien...,54b43265e138239d8684ef96,"[{'type': 'doi', 'id': '10...",article,234--245
9,Wiley,Contrasting pollen histori...,{'name': {'name': 'Journal...,"[{'name': 'Hanselman, Jenn...",2005,7-8,20,[{'url': 'http://doi.wiley...,56f8f62ccf58f1710ac939e0,"[{'type': 'doi', 'id': '10...",article,663--670


## EDA

Reviewing our data includes skimming through some papers online and seeing if the data is consistent with our NLP Sentences dataframe.

From there, we can also visualize what we would like our model to predict: 'Location', 'Site Name', 'Age Span', and 'Site Description' from a "Human perspective".

In [158]:
def order_article(article_id):
    '''
    Function to find an article by its gddid in the NLP sentences and have it displayed in order
    
    Keyword arguments:
    article_id -- gddid
    
    Returns:
    article ordered by sentence index
    '''
    article = nlp_sentences[nlp_sentences['_gddid'] == article_id]
    return article[['sentid', 'words']].sort_values(by = 'sentid')

In [161]:
# Article 01 
order_article('550453fde1382326932d85f7')

,sentid,words
94865,1,"[Quaternary, Science, Reviews, 18, -LRB-, 1999, -RRB-, 945, -RCB-, 960, Palaeoclimate, ,, chronology, and, vegetation, history, of, the, Weichselian, Lateglacial, :, comparative, analysis, of, data, from, three, cores, at, Lago, Grande, di, Monticchio, ,, southern, Italy, B., Huntley, *, ,, W.A...."
94866,2,"[Plant, macrofossil, data, from, one, core, provide, complementary, evidence, of, the, palaeovegetation, .]"
94867,3,"[Quantitative, palaeoclimate, reconstructions, are, made, from, the, pollen, data, using, pollen-climate, response, surfaces, .]"
94868,4,"[The, chronological, framework, for, Lateglacial, events, as, well, as, the, palaeoclimate, are, compared, with, other, sources, of, evidence, of, Lateglacial, palaeoenvironments, .]"
94870,5,"[Surface, conditions, in, the, North, Atlantic, appear, to, be, more, directly, linked, to, climatic, conditions, in, Italy, than, is, the, temperature, in, Greenland, .]"
...,...,...
95325,448,"[Vegetation, history, and, climate, of, the, last, 15,000, years, at, Laghi, di, Monticchio, ,, southern, Italy, .]"
95326,449,"[Quaternary, Science, Reviews, ,, 15, ,, 133, -RCB-, 153, .]"
95327,450,"[Zolitschka, ,, B., ,, &, Negendank, ,, J., F., W., -LRB-, 1996, -RRB-, .]"
95329,451,"[Sedimentology, ,, dating, and, palaeoclimatic, interpretation, of, a, 76.3, ka, record, from, Lago, Grande, di, Monticchio, ,, southern, Italy, .]"


Skimmed info: 
 * `Article Name:` Palaeoclimate_chronology_and_vegetation_history_of_the_Weichselian_Lateglacial
 * `Link`: http://www.sciencedirect.com/science/article/pii/S0277379199000074  
 * `Site Name`:  Laghi di Monticchio
 * `Location`:  40° 56' 40" N, 15° 36' 48" E
 * `Age Span`: 
 * `Site Descriptions`: The record from Lago Grande di Monticchio now adds macrofossil evidence of the presence of relatively boreal tree taxa during the Lateglacial interstadial and provides a chronology based upon annually laminated sediments.

In [162]:
# Article 2
order_article('54b43266e138239d8684efed')

,sentid,words
0,1,"[Available, online, at, www.sciencedirect.com, Quaternary, Research, 69, -LRB-, 2008, -RRB-, 263, --, 275, www.elsevier.com/locate/yqres, Development, of, the, mixed, conifer, forest, in, northern, New, Mexico, and, its, relationship, to, Holocene, environmental, change, R., Scott, Anderson, a, ..."
1,2,"[The, Chihuahueños, Bog, record, extends, to, over, 15,000, cal, yr, BP, .]"
2,3,"[An, Artemisia, steppe, ,, then, an, open, Picea, woodland, grew, around, a, small, pond, until, ca., 11,700, cal, yr, BP, when, Pinus, ponderosa, became, established, .]"
3,4,"[C/N, ratios, ,, δ13C, and, δ15N, values, indicate, both, terrestrial, and, aquatic, organic, matter, was, incorporated, into, the, sediment, .]"
4,5,"[Higher, percentages, of, aquatic, algae, and, elevated, C/N, ratios, indicate, higher, lake, levels, at, the, opening, of, the, Holocene, ,, but, a, wetland, developed, subsequently, as, climate, warmed, .]"
...,...,...
605,572,"[Is, the, Valles, caldera, entering, a, new, cycle, of, activity, ?]"
144,573,"[Geology, 23, ,, 411, --, 414, .]"
607,574,"[Wright, Jr., ,, H.E., ,, Bent, ,, A.M., ,, Hansen, ,, B.S., ,, Maher, Jr., ,, L.J., ,, 1973, .]"
608,575,"[Present, and, past, vegetation, of, the, Chuska, Mountains, ,, northwestern, New, Mexico, .]"


Skimmed info:  
* `Article:` Development of the Mixed Connifer Forest in Northern New Mexico and its relationship to Holocene environmental change
* `Link`: http://www.sciencedirect.com/science/article/pii/S0033589407001512
* `Site Name`:  Chihuahuenos Bog 
* `Location`:  ??? No Given Coordinates 
* `Age Span`:   
* `Site Descriptions`:  Chihuahueños Bog is located within the mixed conifer forest and has affinities to the flora of the southern Rocky Mountains(Foxx and Tierney, 1985; Allen, 2004). Although the area immediately surrounding the bog has been heavily logged in recent years, nearly every conifer species growing in montaneforests of the Jemez mountains is represented either along theshore of the bog or in the surrounding uplands.  

In [163]:
# Article 3
order_article('57c5b941cf58f1338eaddb5b')

,sentid,words
67708,1,"[A, record, of, sustained, prehistoric, and, historic, land, use, from, the, Cahokia, region, ,, Illinois, ,, USA, Samuel, E., Munoz1, ,, Sissel, Schroeder2, ,, David, A., Fike3, ,, and, John, W., Williams1, 1Department, of, Geography, ,, University, of, Wisconsin, --, Madison, ,, 550, North, Pa..."
67709,2,"[Here, we, report, a, high-resolution, and, multiproxy, paleoecological, record, from, Horseshoe, Lake, ,, an, oxbow, lake, in, the, central, Mississippi, River, valley, that, is, adjacent, to, the, Cahokia, site, -LRB-, Illinois, ,, USA, -RRB-, ,, the, largest, prehistoric, settlement, north, o..."
67710,3,"[Palynological, and, carbon, isotope, data, document, pronounced, vegetation, changes, over, the, past, 1700, yr, driven, primarily, by, land, use, ,, including, 900, yr, -LRB-, 450, --, 1350, CE, -RRB-, of, sustained, prehistoric, human, impacts, .]"
67712,4,"[Rapid, forest, clearance, was, followed, closely, by, the, proliferation, of, indigenous, seed, crops, of, the, Eastern, Agricultural, Complex, beginning, ca., 450, CE, ,, centuries, before, the, emergence, of, Cahokia, at, 1050, CE, .]"
67713,5,"[Agricultural, intensiﬁcation, that, included, the, use, of, maize, -LRB-, Zea, mays, subsp, .]"
...,...,...
67845,123,"[Simon, ,, M.L., ,, and, Parker, ,, K.E., ,, 2006, ,, Prehistoric, plant, use, in, the, American, Bottom, :, New, thoughts, and, interpretations, :, Southeastern, Archaeology, ,, v., 25, ,, p., 212, --, 257, .]"
67846,124,"[Smith, ,, B.D., ,, and, Yarnell, ,, R.A., ,, 2009, ,, Initial, formation, of, an, indigenous, crop, complex, in, eastern, North, America, at, 3800, B.P, :, National, Academy, of, Sciences, Proceedings, ,, v., 106, ,, p., 6561, --, 6566, ,, doi, :10.1073, /, pnas, .0901846106, .]"
67848,125,"[Sugita, ,, S., ,, 1993, ,, A, model, of, pollen, source, area, for, an, entire, lake, surface, :, Quaternary, Research, ,, v., 39, ,, p., 239, --, 244, ,, doi, :10.1006, /, qres, .1993.1027, .]"
67849,126,"[Trubitt, ,, M.B.D., ,, 2000, ,, Mound, building, and, prestige, goods, exchange, :, Changing, strategies, in, the, Cahokia, chiefdom, :, American, Antiquity, ,, v., 65, ,, p., 669, --, 690, ,, doi, :10.2307, /, 2694421, .]"


Skimmed info:  
* `Link`: http://dx.doi.org/10.1130/g35541.1  # No Full access to article
* `Site Name`:  Cahokia region, Illinois, USA
* `Location`:  N38°42'00" - N38°42'00", W90°04'60" - W90°04'60" - Not found on text (yet).
* `Age Span`:   
* `Site Descriptions`: Record from Horseshoe Lake, an oxbow lake in the central Mississippi River valley that is adjacent to the Cahokia site (Illinois, USA), the largest prehistoric settlement north of Mexico.   

In [164]:
# Article 4
order_article('58d29193cf58f14928755ba5')

,sentid,words
110659,1,"[Grana, ISSN, :, 0017-3134, -LRB-, Print, -RRB-, 1651-2049, -LRB-, Online, -RRB-, Journal, homepage, :, http://www.tandfonline.com/loi/sgra20, 6, .]"
110660,2,"[Peat-bog, Begbunar, -LRB-, Osogovo, Mountains, ,, southwest, Bulgaria, -RRB-, :, Four, millennia, of, vegetation, history, Maria, Lazarova, ,, Spassimir, Tonkov, ,, Ian, Snowball, &, Elena, Marinova, To, cite, this, article, :, Maria, Lazarova, ,, Spassimir, Tonkov, ,, Ian, Snowball, &, Elena, ..."
110661,3,"[Peat-bog, Begbunar, -LRB-, Osogovo, Mountains, ,, south-west, Bulgaria, -RRB-, :, Four, millennia, of, vegetation, history, ,, Grana, ,, 48:2, ,, 147-149, ,, DOI, :, 10.1080, /, 00173130902965157, To, link, to, this, article, :, http://dx.doi.org/10.1080/00173130902965157, Published, online, :,..."
110662,4,"[Submit, your, article, to, this, journal, Article, views, :, 182, View, related, articles, Citing, articles, :, 5, View, citing, articles, Full, Terms, &, Conditions, of, access, and, use, can, be, found, at, http://www.tandfonline.com/action/journalInformation?journalCode=sgra20, Download, by,..."
110665,5,"[Peat-bog, Begbunar, -LRB-, Osogovo, Mountains, ,, south-west, Bulgaria, -RRB-, :, Four, millennia, of, vegetation, history, MARIAEPDB6.Begbunar, -LRB-, Bulgaria, -RRB-, LAZAROVA1, ,, SPASSIMIR, TONKOV2, ,, IAN, SNOWBALL3, &, ELENA, MARINOVA4, 1Institute, of, Botany, ,, Bulgarian, Academy, of, S..."
...,...,...
110740,80,"[Ann, .]"
110741,81,"[Sofia, Univ., .]"
110742,82,"[Fac, .]"
110743,83,"[Geol, .]"


In [228]:
# Try to see the exact location. Wont show last few words.
pd.DataFrame(order_article('58d29193cf58f14928755ba5').iloc[4])

,110665
sentid,5
words,"[Peat-bog, Begbunar, -LRB-, Osogovo, Mountains, ,, south-west, Bulgaria, -RRB-, :, Four, millennia, of, vegetation, history, MARIAEPDB6.Begbunar, -LRB-, Bulgaria, -RRB-, LAZAROVA1, ,, SPASSIMIR, TONKOV2, ,, IAN, SNOWBALL3, &, ELENA, MARINOVA4, 1Institute, of, Botany, ,, Bulgarian, Academy, of, Sciences, ,, Sofia, ,, Bulgaria, ,, 2Laboratory, of, Palynology, ,, Department, of, Botany, ,, Faculty, of, Biology, ,, Sofia, University, ``, St., Kliment, Ohridski, '', ,, Sofia, ,, Bulgaria, ,, 3GeoBiosphere, Science, Centre, ,, Quaternary, Sciences, ,, Department, of, Geology, ,, Lund, University, ,, Lund, ,, Sweden, ,, 4Center, for, Archaeological, Sciences, ,, Katholieke, Universiteit, Leuven, ,, Leuven, ,, Belgium, Site, details, The, peat-bog, ...]"


Skimmed info:  
* `Link:`http://www.tandfonline.com/doi/abs/10.1080/00173130902965157
* `Site Name`:  Begbunar
* `Location`:  42°09′ N, 22° 33′ E; 1750 ; in SQL file shows as: `42,°,09,cents,N,,,22,°,33,cents,E`
* `Age Span`:   
* `Site Descriptions`: Located in the central treeless zone of the Osogovo mountains, which are situated at the border between south-western Bulgaria and the north-eastern former Yugoslav Republic of Macedonia.

In [235]:
# Article 5
order_article('57928e07cf58f133d1c26609')

,sentid,words
39606,1,"[Timberline, fluctuations, and, late, Quaternary, paleoclimates, in, the, Southern, Rocky, Mountains, ,, Colorado, Patricia, L., Fall, *, Department, of, Geography, ,, Arizona, State, University, ,, Tempe, ,, Arizona, 85287, ABSTRACT, Pollen, and, plant, macrofossils, from, eight, sedimentary, b..."
39607,2,"[By, tracking, climatically, sensitive, forest, boundaries, ,, the, moisturecontrolled, lower, timberline, and, the, temperature-controlled, upper, timberline, ,, paleoclimatic, estimates, can, be, derived, from, modern, temperature, and, precipitation, lapse, rates, .]"
39608,3,"[Pollen, data, suggest, that, prior, to, 11, 000, yr, B.P., ,, a, subalpine, forest, dominated, by, Picea, -LRB-, spruce, -RRB-, and, Pinus, -LRB-, pine, -RRB-, grew, 300Ð700, m, below, its, modern, limit, .]"
39609,4,"[The, inferred, climate, was, 2Ð5, ¡, C, cooler, and, had, 7Ð16, cm, greater, precipitation, than, today, .]"
39611,5,"[Abies, -LRB-, fir, -RRB-, increased, in, abundance, in, the, subalpine, forest, around, 11, 000, yr, B.P., ,, probably, in, response, to, cooler, conditions, with, increased, winter, snow, .]"
...,...,...
40107,448,"[Weber, ,, W., A., ,, 1987, ,, Colorado, flora, :, Western, slope, :, Boulder, ,, Colorado, Associated, University, Press, ,, 530, p., Wells, ,, P., V., ,, 1983, ,, Late-Quaternary, vegetation, of, the, Great, Plains, :, Nebraska, Academy, of, Sciences, Transactions, ,, v., XI, ,, p., 83Ð89, .]"
40108,449,"[Whitlock, ,, C., ,, 1993, ,, Postglacial, vegetation, and, climate, of, Grand, Teton, and, southern, Yellowstone, National, Parks, :, Ecological, Monographs, ,, v., 63, ,, p., 173Ð198, .]"
40110,450,"[Whitlock, ,, C., ,, and, Bartlein, ,, P., J., ,, 1993, ,, Spatial, variation, of, Holocene, climatic, change, in, the, Yellowstone, region, :, Quaternary, Research, ,, v., 39, ,, p., 231Ð238, .]"
40111,451,"[Wright, ,, H., E., ,, Jr., ,, 1983, ,, Late-Quaternary, environments, of, the, United, States, ,, Volume, 2, :, The, Holocene, :, Minneapolis, ,, University, of, Minnesota, Press, .]"


In [233]:
# Print the exact line where locations are.
pd.DataFrame(order_article('57928e07cf58f133d1c26609').iloc[179])

,39801
sentid,180
words,"[LOCATION, AND, AGE, OF, FOSSIL, STUDY, SITES, Core, Location, Lat, Long, -LRB-, °, N, -RRB-, -LRB-, °, W, -RRB-, Elevation, Depth, -LRB-, m, -RRB-, -LRB-, cm, -RRB-, Age, Laboratory, -LRB-, yr, B.P., -RRB-, number, Cottonwood, III, 38, °, 49, ′, 50, ′, ′, 106, °, 24, ′, 45, ′, ′, 3670, 40, --, 50, 80, --, 90, 120, --, 130, 2460, ±, 130, 5790, ±, 180, 8230, ±, 200, A-4481, A-4482, A-4483, Cottonwood, IV, 38, °, 49, ′, 50, ′, ′, 106, °, 24, ′, 45, ′, ′, 3670, 60, --, 80, 5610, ±, 150, A-4487, 148, --, 168, 8790, ...]"


Skimmed info:  
* `Link`: http://dx.doi.org/10.1130/0016-7606(1997)109<1306:tfalqp>2.3.co
* `Site Name`:  Cottonwood III, Cottonwood IV, Red Lady, Red Well, Splains
* `Location`:  38°49'50' 106°24'45', 38°49'50' 106°24'45', 38°52'50' 107°2'30', 38°52'50' 107°2'30',38°53'40' 107°3'15', 38°52'28' 107°05', 38°50' 107°4'30', 38°50' 107°4'30', 38°52' 107°2'30', 38°45' 106°50', 38°45' 106°50'
* `Age Span`:   17000 - 400
* `Site Descriptions`:   The lower timberline or lower forest border is the lower elevational limit of a continuous forest belt. In the Rocky Mountains it forms a sharp contrast with shrublands or grasslands in valley bottoms. Trees may extend below the lower timberline in favorable habitats, along streams or on rocky outcrops.  

In [166]:
# Find the article in the bibliography by gddid to retrieve Article Name

bibliography[bibliography['_gddid'] == '57928e07cf58f133d1c26609']

,publisher,title,journal,author,year,number,volume,link,_gddid,identifier,type,pages
93,GSA,"Timberline fluctuations and late Quaternary paleoclimates in the Southern Rocky Mountains, Colorado",{'name': {'name': 'Geological Society of America Bulletin'}},"[{'name': 'Fall, Patricia L.'}]",1997,10,109,"[{'url': 'http://dx.doi.org/10.1130/0016-7606(1997)109<1306:tfalqp>2.3.co;2', 'type': 'publisher'}]",57928e07cf58f133d1c26609,"[{'type': 'doi', 'id': '10.1130/0016-7606(1997)109<1306:tfalqp>2.3.co;2'}]",article,1306--1320


### Finding Latitude and Longitud via REGEX 

After reviewing some PDF's and viewing the data parsing, I noticed some possible patterns for coordinates.

<img src="figures/img/00_snippet_of_pdf.png" alt="Drawing" style="width: 300px;"/>

Which translates to our pandas dataframe as:

![alt text](figures/img/01_snippet_in_sql.png "PandasVersion")

As a first problem, we can notice that the degrees symbol `°` was translated as a `3`. On some other occassions, it is confused as an `o` or a middle point `◦`. For `'`, this symbol got confused by the word `cents`.

I am still going to use normal REGEX to find coordinates in the most intuitive way. 

A coordinate for longitude is a number ranging from -180° to 180°. It has three components, `°, ',"` and it covers the position in E or W.

Latitude goes from -90° to 90°, has the same three components and goes from S to N.

Using REGEX, and assuming data consistency, the following code should be able to extract coordinates. 

In [103]:
dms_regex = "r'\d\d'"
dfd = "[\\{,]([-]?[1]?[0-9]{1,2}?)(?:(?:,[°◦o],)|(?:[O])|(?:,`{2},))([1]?[0-9]{1,2}(?:.[0-9]*)),[′'`]?[,]?([[0-9]{0,2}]?)[\"]?[,]?([NESWnesw]?),"

dd_regex = "[\\{,][-]?[1]?[0-9]{1,2}\\.[0-9]{1,}[,]?[NESWnesw],"

dfd = 
"[\\{,]([-]?[1]?[0-9]{1,2}?)(?:(?:,[°◦o],)|(?:[O])|(?:,`{2},))([1]?[0-9]{1,2}(?:.[0-9]*)),[′'`]?[,]?([[0-9]{0,2}]?)[\"]?[,]?([NESWnesw]?),"

In [168]:
nlp_sentences['dms_regex'] = nlp_sentences['words'].astype(str).str.findall(r'[-]?\d[0-180]')
                                                                            #'(?:(?:,[°◦o3],)|(?:[O])|(?:,`{2},))')
nlp_sentences['dms_regex']

#nlp_sentences[(nlp_sentences['dms_regex'].astype(str).str.len() > 0)]

0         [20, 08, 60, 11, 60, 60, 11, 28, 20, 20, 08]
1                                                 [00]
2                                             [11, 70]
3                                                   []
4                                                   []
                              ...                     
111960                                              []
111961                                        [61, 61]
111962                                            [20]
111963                                              []
111964                                [20, 10, 10, 10]
Name: dms_regex, Length: 111965, dtype: object

In [205]:
italy_example = nlp_sentences[nlp_sentences['_gddid'] == '550453fde1382326932d85f7']
italy_example = italy_example[italy_example['sentid'] == 10]
italy_example[['words', 'dms_regex']]

,words,dms_regex
94875,"[Introduction, Laghi, di, Monticchio, are, two, maar, lakes, located, in, the, explosion, crater, that, lies, west, of, the, peak, of, Monte, Vulture, ,, near, Mel, '', in, the, Basilicata, region, of, southern, Italy, -LRB-, 403, 56, 40, N, ,, 153, 36, 48, E, -RRB-, -LRB-, Fig., 1, -RRB-, .]","[40, 40, 48]"


### Problems with the REGEX approach

As seen before, extracting information using REGEX is very complicated. We would require a lot of queries that would retrieve each particular case.

A possible alternative could be using NER, Named Entity Recognition.